In [ ]:
import numpy as np
import pandas as pd
import requests
from matplotlib import pyplot as plt

In [ ]:
df1 = pd.read_csv("IDA_data1.csv")
df2 = pd.read_csv("IDA_data2.csv")
df3 = pd.read_csv("IDA_data3.csv")
df1.head()

In [ ]:
df3.head()